# 텍스트를 위한 딥러닝

## 시퀀스 데이터(Sequence data)

* 순서가 있는 데이터
* 특정 순서에 따라 요소들이 배열되어 있다.
    - 순서 정보가 데이터의 중요한 특징
    - 순서가 변경되면 데이터의 의미나 패턴이 변경될 수 있다.

### 시퀀스 데이터의 종류
* **자연어 텍스트**
    - 단어들이 순서대로 나열되어 문장을 형성하며, 문맥과 문법에 따라 의미가 형성된다.
    - 단어 시퀀스, 문자 시퀀스, 문장 시퀀스 등 다양하다.
* **시계열 데이터**
    - 시간 순서에 따라 측정된 값들이 나열됨
    - ex) 주식가격, 기온, 판매량,...

## 텍스트 벡터화(vectorizing)

### 텍스트를 수치형 텐서로 변환하는 과정

* 원시 텍스트를 벡터로 바꾸기 [ 텍스트 → (표준화) → 표준화된 텍스트 → (토큰화) → 토큰 → (인덱싱) → 토큰 인덱스 → (원-핫 인코딩 or 임베딩) → 인덱스의 벡터 인코딩 ]
* 텍스트 표준화: 모델이 인코딩 차이를 고려하지 않도록 제거하기 위한 기초적인 특성 공학의 한 형태(소문자 변환 등)
* 토큰(token): 텍스트를 나누는 단어, 문자, 단위
* 토큰화(tokenization): 텍스트를 토큰으로 나누는 작업: 단어 수준 토큰화, N-그램 토큰화, 문자 수준 토큰화
* 토큰화를 위한 두 종류의 텍스트 처리 모델
    - BoW 모델(N-그램 토큰화 사용)
    - 시퀀스 모델
* 토큰을 수치형 벡터에 연결하는 방법
    - 원핫 인코딩(One-hot encoding)
    - 임베딩(embedding)

## TextVectorization

In [1]:
import string

# 텍스트 벡터화 클래스 생성
class Vectorizer:
    
    # 텍스트 표준화
    def standardize(self, text):
        # 텍스트를 소문자로 변환
        text = text.lower()
        # text의 각 문지 char에 대해 반목하되, char가 string.punctuation(구두점:, 문장부호, 특수기호 등)에 속하지 않는 경우에만 그대로 유지
        # join을 이용하여 하나의 문자열로 결합
        return "".join(char for char in text if char not in string.punctuation)
    
    def tokenize(self, text):
        # 공백을 기준으로 분리
        return text.split()
    
    # 토큰에 대한 어휘사전 생성
    def make_vocabulary(self, dataset):
        # 어휘사전의 첫 두 항목은 공백(0)과 unknown(1)
        # self.vocabulary 변수는 텍스트 데이터셋에서 생성된 어휘사전을 저장하는 변수
        # 초기값으로는 빈 문자열과("") unknown('[UNK]')을 각각 0과 1의 인덱스로 가짐
        self.vocabulary = {"": 0, "[UNK]": 1}
        # dataset 내 문자열에 대하여 반복
        for text in dataset:
            # 표준화
            text = self.standardize(text)
            
            # 공백 기준 토큰화
            tokens = self.tokenize(text)
            
            # 어휘사전에 token이 없으면 새로운 단어로 판단하여 'self.vocabulary'에 추가
            # 'self.vocabulary'의 현재 크기, 즉 추가된 단어의 개수를 이 단어의 인덱스 값으로 할당
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
                    
        # 생성된 어휘사전의 key와 value를 바꾼 inverse_vocabulary 생성, 정수 인덱스: 토큰
        self.inverse_vocabulary = dict(
            # 'self.vocabulary.items()' 메소드를 사용하여 'self.vocabulary' 사전의 모든 (key, value) 쌍을 가져와
            # 이를 반대로 뒤집은 새로운 (value, key) 쌍을 생성
            (v, k) for k, v in self.vocabulary.items())
        
        
    # 전달된 텍스트를 표준화, 토큰화를 한 후 각 토큰에 대하며 어휘사전에 해당하는 정수 인덱스를 찯아서 대체함
    def encode(self, text):
        # 표준화
        text = self.standardize(text)
        # 토큰화
        tokens = self.tokenize(text)
        # 정수 인덱스를 찾아 대체
        return [self.vocabulary.get(token, 1) for token in tokens]
    
    
    # inverse_vocabulary를 사용하여 인코더와 반대로 전달된 정수 시퀀스를 원래 텍스트로 복원
    def decode(self, int_sequence):
        return " ".join(self.inverse_vocabulary.get(i, '[UNK]') for i in int_sequence)
    

In [2]:
# 클래스 객체 생성
vectorizer = Vectorizer()

# 데이터로 사용할 수 있는 텍스트 변수 정의
dataset = [
    'I write, erase, rewrite',
    'Erase again, and then',
    'A poppy blooms.',
]

# make_vocabulary 메소드 실행
vectorizer.make_vocabulary(dataset)

In [3]:
# 만들어진 어휘사전으로 새로운 텍스트 시퀀스 인코딩
test_sentence = 'I write, rewrite, and still rewrite again'
# test_sentence를 vectorizer.encode()를 이용하여 정수 시퀀스로 인코딩하여 변수에 저장
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)

[2, 3, 5, 7, 1, 5, 6]


In [4]:
# test_sentence_1
test_sentence_test1 = 'I write, erase, rewrite'
encoded_sentence_test1 = vectorizer.encode(test_sentence_test1)
print(encoded_sentence_test1)

[2, 3, 4, 5]


In [5]:
# test_sentence_2
test_sentence_test2 = 'Erase again, and then.'
encoded_sentence_test2 = vectorizer.encode(test_sentence_test2)
print(encoded_sentence_test2)

[4, 6, 7, 8]


In [6]:
# test_sentence_3
test_sentence_test3 = 'I write, erase and delete'
encoded_sentence_test3 = vectorizer.encode(test_sentence_test3)
print(encoded_sentence_test3)

[2, 3, 4, 7, 1]


In [7]:
# test_sentence_4
test_sentence_test4 = 'I like apple, and also banana'
encoded_sentence_test4 = vectorizer.encode(test_sentence_test4)
print(encoded_sentence_test4)

[2, 1, 1, 7, 1, 1]


* 본 예제에서는 공백을 기준으로 토큰화를 진행하였으므로, 빈 문자열인 0의 경우는 없게 된다.

In [8]:
# 위의 인코딩 값을 디코딩(다시 텍스트로 복원)
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


In [9]:
decoded_sentence_test1 = vectorizer.decode(encoded_sentence_test1)
print(decoded_sentence_test1)

i write erase rewrite


In [10]:
decoded_sentence_test2 = vectorizer.decode(encoded_sentence_test2)
print(decoded_sentence_test2)

erase again and then


In [11]:
decoded_sentence_test3 = vectorizer.decode(encoded_sentence_test3)
print(decoded_sentence_test3)

i write erase and [UNK]


In [12]:
decoded_sentence_test4 = vectorizer.decode(encoded_sentence_test4)
print(decoded_sentence_test4)

i [UNK] [UNK] and [UNK] [UNK]


위의 방식은 성능이 좋지 않다. 따라서 빠르고 효율적인 keras의 TextVectorization을 사용한다.

## 케라스에서 제공하는 텍스트 벡터화 레이어 적용

In [13]:
from keras.layers import TextVectorization

# 객체 생성
# 정수 인덱스로 인코딩 되도록 설정
text_vectorization = TextVectorization(output_mode = 'int')

In [14]:
# 정규표현식(Regular Expression)을 사용하여 문자열을 처리하는 라이브러리
import re
# 문자열 관련 유틸리티 함수 제공
import string
import tensorflow as tf

# 텍스트 표준화 사용자 함수 작성
def custom_standardization_fn(string_tensor):
    # string_tensor 내 모든 문자열을 소문자로 변환
    lowercase_string = tf.strings.lower(string_tensor)
    
    # tf.strings.regex_replace는 input, pattern, rewrite를 인자로 받는다
    # re.escape(): 특수문자 등을 escape(삭제)하여 정규표현식에서 사용할 수 있는 형태로 만들어 줌
    # f'[{re.escape(string.punctuation)}]': 구두점(string.punctuation)을 찾아 정규표현식 패턴으로 만듦
    # 입력문 lowercase_string에서 구두점 문자를 찾아 ""로 변환하여 반환
    return tf.strings.regex_replace(lowercase_string, f'[{re.escape(string.punctuation)}]', "")

# 문자열을 공백 기준으로 분할
def custom_split_fn(string_tensor):
    # 문자열을 분리
    return tf.strings.split(string_tensor)

# 텍스트 벡터화 레이어 생성
text_vectorization = TextVectorization(
    output_mode = 'int',      # 출력결과의 형태
    standardize = custom_standardization_fn,       # 텍스트 표준화 함수 설정: 위의 사용자 함수를 사용
    split = custom_split_fn,        # 토큰화 함수 설정
)

In [15]:
# tf.strings.regex_replace() 더 살펴보기
test_text = "GOOD~GOOD~GOOD~GOOD!!"

In [16]:
m = re.escape('~')
m

'\\~'

In [17]:
test_text_2 = tf.strings.regex_replace(test_text, m, "")
test_text_2

<tf.Tensor: shape=(), dtype=string, numpy=b'GOODGOODGOODGOOD!!'>

* 텍스트 말뭉치의 어휘 사전을 인덱싱하기 위해 문자열을 반환하는 Dataset 객체로 이 층의 adapt()메서드를 호출한다.

In [18]:
# input data 정의
dataset = [
    'I write, erase, rewrite',
    'Erase again, and then',
    'A poppy blooms.',
]
# TextVectorization 클래스 객체가 현재 데이터셋에 맞게 적용(adapt)되어, 어휘 사전 생성
text_vectorization.adapt(dataset)

### 어휘 사전 출력하기

In [19]:
# 어휘 사전 출력(0은 공백, 1은 unknown)
# TextVectorization 클래스를 사용하여 생성된 어휘 사전(vocabulary)을 조회하는 메소드
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

* 앞선 클래스 정의때와는 달리, 케라스의 텍스트 벡터화 레이어의 어휘사전은 가장 빈도가 많은 단어, 알파벳의 역순으로 정렬이 된다.

In [20]:
# 인코딩

# 어휘 사전을 vocabulary 변수에 할당
vocabulary = text_vectorization.get_vocabulary()

# 새로운 input data 정의
test_sentence = 'I write, rewrite, and still rewrite again'

# 벡터화 레이어에 새로운 input data를 투입하여 인코딩된 정수 시퀀스 반환
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [21]:
# 디코딩
# inverse_vocabulary 생성
inverse_vocab = dict(enumerate(vocabulary))

# enumerate()는 리스트를 입력값으로 받아서 인덱스, 값을 쌍으로 반환(인덱스는 0부터 시작한다)
# dict()는 key-value 쌍을 입력으로 받아서, 딕셔너리 생성
# dict(enumerate(vocabulary)): 각 단어에 대한 인덱스를 키(key)로,  각 단어를 값(value)으로 저장하므로
# 이 딕셔너리를 어휘 사전의 inverse(역사전)으로 사용

# 정수 시퀀스를 다시 텍스트화
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


# 토큰화를 위한 두 종류의 텍스트 처리 모델

## IMDB 영화 리뷰 데이터 다운로드

In [22]:
# 데이터 다운로드
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# 압축 해제
!tar -xf aclImdb_v1.tar.gz
# unsup 폴더 삭제
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 80.2M    0 32768    0     0  31256      0  0:44:51  0:00:01  0:44:50 31297
  0 80.2M    0  368k    0     0   177k      0  0:07:42  0:00:02  0:07:40  177k
  1 80.2M    1  896k    0     0   297k      0  0:04:36  0:00:03  0:04:33  297k
  1 80.2M    1 1616k    0     0   401k      0  0:03:24  0:00:04  0:03:20  401k
  3 80.2M    3 2768k    0     0   552k      0  0:02:28  0:00:05  0:02:23  553k
  5 80.2M    5 4624k    0     0   763k      0  0:01:47  0:00:06  0:01:41  917k
  8 80.2M    8 7216k    0     0  1030k      0  0:01:19  0:00:07  0:01:12 1387k
 13 80.2M   13 11.1M    0     0  1420k      0  0:00:57  0:00:08  0:00:49 2098k
 21 80.2M   21 17.4M    0     0  1983k      0  0:00

In [23]:
# 데이터 샘플 확인
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy


In [24]:
import os, pathlib, shutil, random

# 기본 경로 설정
base_dir = pathlib.Path('aclImdb')

# 검증 데이터 경로 설정
val_dir = base_dir / 'val'

# 학습 데이터 경로 설정
train_dir = base_dir / 'train'

# 'neg'와 'pos' 두 카테고리에 대해 반복문을 실행
# 'neg': 부정적 리뷰, 'pos': 긍정적 리뷰
for category in ('neg', 'pos'):
    # 각 카테고리에 대한 검증 데이터셋 디렉토리를 생성
    os.makedirs(val_dir / category)
    
    # 현재 카테고리의 훈련 데이터셋 디렉토리에서 모든 파일 목록을 가져와 files 변수에 저장
    files = os.listdir(train_dir / category)
    
    # 시드값 1337으로 files 변수에 저장된 파일들을 무작위로 섞음
    random.Random(1337).shuffle(files)
    
    # 훈련 데이터셋에서 20%를 검증 데아터로 사용
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    
    # 추출된 검증 데이터 파일들을 순회
    for fname in val_files:
        # 각 파일을 훈련 데이터셋 디렉토리에서 검증 데이터셋 디렉토리로 이동
        shutil.move(train_dir / category / fname, val_dir / category / fname)

In [25]:
from tensorflow import keras

# 배치 사이즈를 32로 설정
batch_size = 32

# 학습 데이터셋 로드
train_ds = keras.utils.text_dataset_from_directory('aclImdb/train', batch_size = batch_size)

# 검증 데이터셋 로드
val_ds = keras.utils.text_dataset_from_directory('aclImdb/val', batch_size = batch_size)

# 평가 데이터셋 로드
test_ds = keras.utils.text_dataset_from_directory('aclImdb/test', batch_size = batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [26]:
# 첫번째 배치의 크기와 dtype 출력하기
for inputs, targets in train_ds:
    # 입력 데이터의 형태
    print('inputs.shape: ', inputs.shape)
    # 입력 데이터의 데이터 타입
    print('inputs.dtype: ', inputs.dtype)
    # 타겟 데이터의 형태
    print('targets.shape: ', targets.shape)
    # 타겟 데이터의 데이터 타입
    print('targets.dtype: ', targets.dtype)
    # 입력 데이터의 첫 번째 샘플 출력
    print('inputs[0]: ', inputs[0])
    # 타겟 데이터의 첫 번째 샘플 출력
    print('targets[0]: ', targets[0])
    break

inputs.shape:  (32,)
inputs.dtype:  <dtype: 'string'>
targets.shape:  (32,)
targets.dtype:  <dtype: 'int32'>
inputs[0]:  tf.Tensor(b"It Could Have Been A Marvelous Story Based On The Ancient Races Of Cat People, but it wasn't.<br /><br />This work could have been just that; marvelous and replete with mythological references which kept my fascination fueled. The lead characters (Charles Brady played by Brian Krause; and his mother Mary, played by Alice Krige) were shallowly done, had no depth of personality and were hardly likable or drawing. Not even M\xc3\xa4dchen Amick (who played Tanya Robertson)'s character fit into that description. <br /><br />However, as I've said many times before, when you adapt a Stephen King novel for TV, you simply must take into account the fact that his books aren't written for TV, and his screenplay talent sadly lacks the fire and depth he exhibits as a novelist. <br /><br />This is another botched attempt to take the magick of Stephen King writing, whet

## 단어를 집합으로 처리하기: BoW(Bag of Word)방식

* Text: 'the cat sat on the mat'
    - 1-gram(unigram): ['the', 'cat', 'sat', 'on', 'the', 'mat']
    - 2-gram(bi-grams): ['the cat', 'cat sat', 'sat on', 'on the', 'the mat']
    - 3-gram(tri-grams): ['the cat sat', 'cat sat on', 'sat on the', 'on the mat']
    - 4-gram ....

* 이런 종류의 단어 집합을 사용한 토큰화 방법을 Bag of Word라고 한다.
    - 생성된 토큰은 시퀀스가 아니라 집합이며, 문장의 일반적인 구조가 사라진다.
    - 단순한 집합을 담은 가방을 사용한다고 하여 Bag of Word이라는 용어로 불린다.

### Single words (unigrams) with binary encoding

### TextVectorization 층으로 데이터 전처리하기

In [27]:
from keras.layers import TextVectorization

# 텍스트 벡터화 레이어 생성
text_vectorization = TextVectorization(
    max_tokens = 20000,      # 토큰 최대값(문장 내의 최대 단어 수, 입력값의 길이를 제한)
    output_mode = 'multi_hot',    # 단어의 개수만큼 차원을 가진 벡터로 인코딩
)

# 학습 데이터셋(train_ds)에서 텍스트 데이터만 추출하여 text_only_train_ds에 저장
# train_ds 데이터셋에 대해 map함수를 적용하여 모든 샘플에서 x값만 추출
text_only_train_ds = train_ds.map(lambda x, y: x)

# text_omly_train_ds로 어휘사전 생성
text_vectorization.adapt(text_only_train_ds)

# 학습데이터 셋의 텍스트 데이터에 어휘사전을 적용한 후 저장
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)
# 검증데이터 셋의 텍스트 데이터에 어휘사전을 적용한 후 저장
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)
# 퍙가데이터 셋의 텍스트 데이터에 어휘사전을 적용한 후 저장
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)

### 모델 생성 사용자 함수 작성

In [28]:
from tensorflow import keras
from keras import layers
# 토큰 최대값, 은닉차원의 수는 16로 설정
def get_model(max_tokens = 20000, hidden_dim = 16):
    
    # 함수형 API를 사용한 레이어 생성
    # 입력 텐서 정의
    inputs = keras.Input(shape = (max_tokens, ))
    # 은닉 레이어
    x = layers.Dense(hidden_dim, activation = 'relu')(inputs)
    # 드롭아웃 레이어
    x = layers.Dropout(0.5)(x)
    # 출력 레이어
    outputs = layers.Dense(1, activation = 'sigmoid')(x)
    # 모델 생성
    model = keras.Model(inputs, outputs)
    
    # 모델 캄파일
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

### 이진 유니그램 데이터 학습하고 평가하기

In [29]:
# 모델 생성
model = get_model()

# 모델 요약
model.summary()

# 사용자 콜백 생성
callbacks = [
    keras.callbacks.ModelCheckpoint('binary_1gram.keras', save_best_only = True)
]

# 모델 학습
# cache(): 데이터셋을 메모리에 캐시하여 데이터 처리 과정에서 발생하는 오버헤드 감소(작은 데이터에서 사용함)
model.fit(binary_1gram_train_ds.cache(),
         validation_data = binary_1gram_val_ds.cache(), epochs = 10, callbacks = callbacks)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 13s 19ms/step - loss: 0.3973 - accuracy: 0.8321 - val_loss: 0.3001 - val_accuracy: 0.8794
Epoch 2/10
625/625 [=======================

In [30]:
# 콜백에서 저장한 best 모델 로드
model = keras.models.load_model('binary_1gram.keras')

In [31]:
# 평가 지표 출력
print(f'테스트 정확도: {model.evaluate(binary_1gram_test_ds)[1]:.3f}')

782/782 [==============================] - 26s 32ms/step - loss: 0.2907 - accuracy: 0.8864
테스트 정확도: 0.886


### Bi-grams with binary encoding

### 바이그램을 반환하는 TextVectorization 층 만들기

In [32]:
# 텍스트 벡터화 레이어 생성
# 바이그램이므로 ngrams는 2로 설정
text_vectorization = TextVectorization(ngrams = 2, max_tokens = 20000, output_mode = 'multi_hot')

In [33]:
# 어휘 사전 생성
text_vectorization.adapt(text_only_train_ds)

# 학습데이터 셋의 텍스트 데이터에 어휘사전을 적용한 후 저장
binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)
# 검증데이터 셋의 텍스트 데이터에 어휘사전을 적용한 후 저장
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)
# 평가데이터 셋의 텍스트 데이터에 어휘사전을 적용한 후 저장
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls = 4)

# 모델 생성
model = get_model()

# 모델 요약
model.summary()

# 사용자 콜백 작성
callbacks = [
    keras.callbacks.ModelCheckpoint('binary_2gram.keras', save_best_only = True)
]

# 모델 학습
model.fit(binary_2gram_train_ds.cache(), validation_data = binary_2gram_val_ds.cache(), epochs = 10, callbacks = callbacks)

# best 모델 로드
model = keras.models.load_model('binary_2gram.keras')

# 평가 지표 출력
print(f'테스트 정확도: {model.evaluate(binary_2gram_test_ds)[1]:.3f}')

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 10s 15ms/step - loss: 0.3734 - accuracy: 0.8463 - val_loss: 0.2789 - val_accuracy: 0.8888
Epoch 2/10
625/625 [=====================